In [1]:
import os
import tempfile
from modelscope.msdatasets import MsDataset
from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.hub.snapshot_download import snapshot_download
from modelscope.utils.constant import ModelFile

2024-06-24 22:03:39,825 - modelscope - INFO - PyTorch version 2.3.0+cu121 Found.
2024-06-24 22:03:39,829 - modelscope - INFO - TensorFlow version 2.16.1 Found.
2024-06-24 22:03:39,830 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-06-24 22:03:39,831 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-24 22:03:39,908 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 03830e53bae2c0a8f8ac72c969ac6ac1 and a total number of 980 components indexed
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'damo/cv_ddsar_face-detection_iclr23-damofd'
ms_ds_widerface = MsDataset.load('wider_face_custom', namespace='gaosheng')

data_path = ms_ds_widerface.config_kwargs['split_config']
train_dir = data_path['train']
val_dir = data_path['validation']

def get_name(dir_name):
    names = [i for i in os.listdir(dir_name) if not i.startswith('_')]
    return names[0]

train_root = train_dir + '/' + get_name(train_dir) + '/'
val_root = val_dir + '/' + get_name(val_dir) + '/'
cache_path = snapshot_download(model_id, cache_dir = './wider2')
tmp_dir = tempfile.TemporaryDirectory().name
pretrain_epochs = 640
ft_epochs = 1
total_epochs = pretrain_epochs + ft_epochs
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

def _cfg_modify_fn(cfg):
    cfg.checkpoint_config.interval = 1
    cfg.log_config.interval = 10
    cfg.evaluation.interval = 1
    cfg.data.workers_per_gpu = 1
    cfg.data.samples_per_gpu = 4
    return cfg
kwargs = dict(
    cfg_file=os.path.join(cache_path, 'DamoFD_lms.py'),
    save_pretrained=True,
    work_dir="./wider",
    train_root=train_root,
    model_dir = '.wider',
    val_root=val_root,
    resume_from=os.path.join(cache_path, ModelFile.TORCH_MODEL_FILE),
    total_epochs=total_epochs,  # run #epochs
    cfg_modify_fn=_cfg_modify_fn)

trainer = build_trainer(name=Trainers.face_detection_scrfd, default_args=kwargs)
trainer.train()

2024-06-24 22:03:44,527 - modelscope - INFO - No subset_name specified, defaulting to the default
2024-06-24 22:03:45,043 - modelscope - INFO - Generating dataset wider_face_custom (/mnt/workspace/.cache/modelscope/hub/datasets/modelscope___wider_face_custom/master/train_validation)


100% 

2024-06-24 22:03:48,027 - modelscope - WARNING - Model revision not specified, use revision: v1.1
Downloading: 100%|██████████| 235/235 [00:00<00:00, 672B/s]
Downloading: 100%|██████████| 5.82k/5.82k [00:00<00:00, 17.6kB/s]
Downloading: 100%|██████████| 2.17M/2.17M [00:00<00:00, 5.78MB/s]
Downloading: 100%|██████████| 8.10k/8.10k [00:00<00:00, 23.5kB/s]
/usr/local/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
fatal: not a git repository (or any parent up to mount point /mnt)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
2024-06-24 22:03:55,108 - mmdet - INFO - Environment info:
--------------------

origin image size 313


2024-06-24 22:03:58,918 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-06-24 22:03:58,931 - mmdet - INFO - load checkpoint from local path: ./wider2/damo/cv_ddsar_face-detection_iclr23-damofd/pytorch_model.pt
2024-06-24 22:03:59,032 - mmdet - INFO - the iteration number is changed due to change of GPU number
2024-06-24 22:03:59,035 - mmdet - INFO - resumed epoch 640, iter 1031680
2024-06-24 22:03:59,037 - mmdet - INFO - Start running, host: root@eais-bjtyd0pwdtv28jwgqq9n-98c7c6994-xdxkm, work_dir: /mnt/workspace/wider
2024-06-24 22:03:59,038 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                

origin image size 135


/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /torch/pytorch/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:333: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/usr/local/lib/python3.10/site-packages/mmdet/core/anchor/anchor_generator.py:369: UserWarning: ``single_level_grid_anchors`` would be deprecate

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 135/135, 33.0 task/s, elapsed: 4s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-06-24 22:04:20,236 - mmdet - INFO - 
+-------+-----+------+--------+-------+
| class | gts | dets | recall | ap    |
+-------+-----+------+--------+-------+
| FG    | 162 | 8873 | 1.000  | 0.020 |
+-------+-----+------+--------+-------+
| mAP   |     |      |        | 0.020 |
+-------+-----+------+--------+-------+
2024-06-24 22:04:20,330 - mmdet - INFO - Exp name: DamoFD_0.5g_lms.py
2024-06-24 22:04:20,331 - mmdet - INFO - Epoch(val) [641][135]	AP50: 0.0200, mAP: 0.0199
